In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display, Markdown

from pymedphys.logfile import (
    get_mappings, get_mu_densities_for_file_hashes
)

from pymedphys.msq import mosaiq_connect
from pymedphys.utilities import get_index

from pymedphys.plt import pcolormesh_grid

In [ ]:
config = {
    "linac_logfile_data_directory": "S:\\Physics\\Programming\\data\\LinacLogFiles",
    "machine_map": {
    "2619": {
          "centre": "rccc",
          "type": "elekta-agility"
        },
    "2694": {
          "centre": "rccc",
          "type": "elekta-agility"
        }
    },
    "centres": {
        "rccc": {
          "timezone": "Australia/Sydney",
          "ois": "mosaiq",
          "ois_specific_data": {
            "sql_server": "msqsql:1433"
          }
        }
    },
    "machine_types": {
    "elekta-agility": {
      "gantry_tolerance": 0.5
    }
  },
}

In [ ]:
index = get_index(config)
file_hashes = np.array(list(index.keys()))

field_types = np.array([
    index[file_hash]['delivery_details']['field_type']
    for file_hash in file_hashes
])

file_hashes = file_hashes[field_types == 'DMLC']

is_qa = np.array([
    index[file_hash]['delivery_details']['qa_mode']
    for file_hash in file_hashes
])

file_hashes = file_hashes[np.invert(is_qa)]

machine = np.array([
    index[file_hash]['logfile_header']['machine']
    for file_hash in file_hashes
])

# limit to RCCC for now
file_hashes = file_hashes[(machine == '2619') | (machine == '2694')]
np.random.shuffle(file_hashes)

# might need course information
# index[file_hashes[0]]


In [ ]:
# filepath = get_filepath(index, config, file_hash)

In [ ]:
patient_grouped_fields, field_id_grouped_hashes = get_mappings(index, file_hashes)

In [ ]:
# patient_id = ''
# cache = {}
# field_ids = patient_grouped_fields[patient_id]

# field_ids

In [ ]:
field_id = 83306

In [ ]:
file_hashes = np.array(field_id_grouped_hashes[field_id])

local_time = np.array([
    index[file_hash]['local_time']
    for file_hash in file_hashes
]).astype(np.datetime64)

df = pd.DataFrame(data={'local_time': local_time, 'file_hashes': file_hashes})

df = df.sort_values('local_time')
df

In [ ]:
date_to_check = '2019-02-21 09:49'
surrounding_number_of_hours = 48

date = np.array(date_to_check, dtype=np.datetime64)
lookup = np.abs(df['local_time'] - date) < np.timedelta64(surrounding_number_of_hours, 'h')

file_hashes = df[lookup]['file_hashes'].values
file_hashes

In [ ]:
with mosaiq_connect('msqsql') as cursor:
    mu_densities = get_mu_densities_for_file_hashes(index, config, cursor, file_hashes)

In [ ]:
mosaiq, logfile, normalisation = mu_densities

In [ ]:
gantry_angles = list(mosaiq.keys())
gantry_angles

In [ ]:
figsize = (15, 5)

def plot_mu_density(logfile_mu_density, mosaiq_mu_density):
    grid_xx = logfile_mu_density[0]
    grid_yy = logfile_mu_density[1]

    logfile_mu_density = logfile_mu_density[2]
    mosaiq_mu_density = mosaiq_mu_density[2]
    
    min_val = np.min([logfile_mu_density, mosaiq_mu_density])
    max_val = np.max([logfile_mu_density, mosaiq_mu_density])
    
    x = grid_xx[0,:]
    y = grid_yy[:,0]

    x, y = pcolormesh_grid(x, y)

    x = -x
    
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=figsize);
    
    plt.figure(figsize=figsize)
    im1 = ax1.pcolormesh(x, y, logfile_mu_density, vmin=min_val, vmax=max_val)
    fig.colorbar(im1, ax=ax1, label='MU density')
    ax1.set_title('Logfile MU density')
    ax1.set_xlabel('MLC direction (mm)')
    ax1.set_ylabel('Jaw direction (mm)')

    ax1.axis('equal')
    
    im2 = ax2.pcolormesh(x, y, mosaiq_mu_density, vmin=min_val, vmax=max_val)
    fig.colorbar(im2, ax=ax2, label='MU density')
    ax2.set_title('Mosaiq MU density')
    ax2.set_xlabel('MLC direction (mm)')
    ax2.set_ylabel('Jaw direction (mm)')

    ax2.axis('equal')
    
    difference = logfile_mu_density - mosaiq_mu_density
    max_range = np.max(np.abs(difference))
    
    im3 = ax3.pcolormesh(x, y, difference, vmin=-max_range, vmax=max_range, cmap='bwr')
    fig.colorbar(im3, ax=ax3, label='Logfile - Mosaiq MU density')
    ax3.set_title('MU density difference')
    ax3.set_xlabel('MLC direction (mm)')
    ax3.set_ylabel('Jaw direction (mm)')

    ax3.axis('equal');

In [ ]:
for logfile_group in list(logfile.keys()):
    for file_hash in logfile_group:
        time_stamp = index[file_hash]['local_time']
        display(Markdown('## Time Stamp = {}'.format(time_stamp)))

    for gantry_angle in gantry_angles:
        display(Markdown('### Gantry Angle = {}'.format(gantry_angle)))
        mosaiq_mu_density = mosaiq[gantry_angle]
        logfile_mu_density = logfile[logfile_group][gantry_angle]

        plot_mu_density(logfile_mu_density, mosaiq_mu_density);

        plt.show()